In [80]:
with open("input.txt") as f:
    lines = [line.strip() for line in f.readlines()]

In [81]:
disk_map = lines[0]

In [82]:
class File:
    def __init__(self, id: int, location: list[int]):
        self.id = id
        self.location = location

    @property
    def size(self) -> int:
        return len(self.location)

    def checksum(self) -> int:
        return self.id * sum(self.location)

    def __repr__(self):
        return f"{self.id}: {self.location}"

### Part 1

In [83]:
files = []
free_spaces = []

current_location = 0

for i in range(len(disk_map) // 2):
    file_size, free_space = int(disk_map[2*i]), int(disk_map[2*i+1])
    files.append(
        File(id=i, location=list(range(current_location, current_location + file_size)))
    )
    current_location = current_location + file_size
    free_spaces.extend(list(range(current_location, current_location + free_space)))
    current_location = current_location + free_space
# Last file:
file_size = int(disk_map[-1])
files.append(
    File(id=len(disk_map) // 2, location=list(range(current_location, current_location + file_size)))
)

In [84]:
done = False

for file in reversed(files):
    new_file_loc = []
    if free_spaces:
        for free_space, file_loc in zip(free_spaces, reversed(file.location)):
            if free_space <= file_loc:
                new_file_loc.append(free_space)
            else:
                new_file_loc.append(file_loc)
                done = True
        file.location = new_file_loc
        if len(free_spaces) >= file.size:
            free_spaces = free_spaces[file.size:]
        else:
            free_spaces = []
    else:
        done = True
    if done:
        break


In [85]:
sum([file.checksum() for file in files])

6283170117911

### Part 2

In [86]:
files = []
free_spaces = {i: [] for i in range(10)}

current_location = 0

for i in range(len(disk_map) // 2):
    file_size, free_space = int(disk_map[2*i]), int(disk_map[2*i+1])
    files.append(
        File(id=i, location=list(range(current_location, current_location + file_size)))
    )
    current_location = current_location + file_size
    free_spaces[free_space].append(current_location)
    current_location = current_location + free_space
# Last file:
file_size = int(disk_map[-1])
files.append(
    File(id=len(disk_map) // 2, location=list(range(current_location, current_location + file_size)))
)

In [87]:
for file in reversed(files):
    possible_new_locations = []
    for i in range(file.size, 10):
        if free_spaces[i]:
            possible_new_locations.append((i, free_spaces[i][0]))
    if possible_new_locations:
        new_location = min(possible_new_locations, key = lambda x: x[1])
        if new_location[1] < file.location[0]:
            prev_free_space, loc_index = new_location
            file.location = list(range(loc_index, loc_index + file.size))
            if prev_free_space > file.size:
                free_spaces[prev_free_space - file.size].append(loc_index + file.size)
                free_spaces[prev_free_space - file.size] = sorted(free_spaces[prev_free_space - file.size])
            free_spaces[prev_free_space].remove(loc_index)

In [88]:
sum([file.checksum() for file in files])

6307653242596